# Выгрузка отзывов из google maps в учебных целях, для использования в pet проекте

In [69]:
from selenium import webdriver
import time
import pandas as pd

In [70]:
chrome_driver_path = r'D:/Документы/Python/Площадки/chromedriver' # путь к драйверу


In [71]:
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(executable_path=chrome_driver_path,options=options)


C:\Users\keybi\AppData\Local\Temp\ipykernel_17828\3194083821.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=chrome_driver_path,options=options)


options = Options() 
options.add_argument("--lang=ru") 
driver = webdriver.Chrome(chrome_options=options)

In [72]:
url = 'https://www.google.com/maps/place/%D0%94%D0%B5%D1%82%D1%81%D0%BA%D0%B0%D1%8F+%D0%B8%D0%B3%D1%80%D0%BE%D0%B2%D0%B0%D1%8F+%D0%BF%D0%BB%D0%BE%D1%89%D0%B0%D0%B4%D0%BA%D0%B0+%22%D0%A1%D0%B0%D0%BB%D1%8E%D1%82%22/@55.7301648,37.6066522,651m/data=!3m1!1e3!4m7!3m6!1s0x46b54b7eea601d5f:0x7ffaa16d311ca514!8m2!3d55.7301648!4d37.6066522!9m1!1b1'
driver.get(url)
time.sleep(5)

## Настройка прокрутки

In [50]:
SCROLL_PAUSE_TIME = 5

# Получить высоту прокрутки
last_height = driver.execute_script("return document.body.scrollHeight")

number = 0

while True:
    number = number+1

    # Прокрутить вниз  
    
    ele = driver.find_element('xpath', '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div[10]')
    driver.execute_script('arguments[0].scrollBy(0, 20);', ele)

    # Ожидание загрузки страницы 

    time.sleep(SCROLL_PAUSE_TIME)

    # Рассчитываем новую высоту прокрутки и сравниваем ее с последней высотой прокрутки 
    print(f'last height: {last_height}')

    ele = driver.find_element('xpath','//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div[10]')

    new_height = driver.execute_script("return arguments[0].scrollHeight", ele)

    print(f'new height: {new_height}')

    if number == 5:
        break

    if new_height == last_height:
        break

    print('cont')
    last_height = new_height

last height: 924
new height: 1801
cont
last height: 1801
new height: 1801


## После прокрутки нужно открыть все «Еще» под отзывами. В противном случае отзыв будет не полным

In [63]:
item = driver.find_elements('xpath','//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]') #xpath тела отзывов

time.sleep(3)

In [64]:
name_list = []
stars_list = []
review_list = []
duration_list = []

In [65]:
for i in item:

    button = i.find_elements(By.TAG_NAME, 'button')
    for m in button:
        if m.text == "Ещё":
            m.click()
    time.sleep(5)

    name = i.find_elements(By.CLASS_NAME, "d4r55")
    stars = i.find_elements(By.CLASS_NAME,"kvMYJc")
    review = i.find_elements(By.CLASS_NAME,"wiI7pd")
    duration = i.find_elements(By.CLASS_NAME,"rsqaWe")

    for j,k,l,p in zip(name,stars,review,duration):
        name_list.append(j.text)
        stars_list.append(p.text)
        review_list.append(k.get_attribute("aria-label"))
        duration_list.append(l.text)

## преобразовываю выводы в файл .csv.

In [66]:
review = pd.DataFrame(
    {'name': name_list,
     'rating': stars_list,
     'review': review_list,
     'duration': duration_list})

review.to_csv('google_review.csv',index=False)
print(review)

                    name            rating     review  \
0                Дмитрий   9 месяцев назад   5 звезд    
1            Михаил Лось  11 месяцев назад   5 звезд    
2          Vika Andreeva         год назад   5 звезд    
3     Светлана Рабинович         год назад   5 звезд    
4           Игорь Тухкин         год назад   5 звезд    
..                   ...               ...        ...   
150         Мария Репина      2 года назад   5 звезд    
151  Кристина Емельянова      2 года назад   5 звезд    
152     Владимир Дедюхин      2 года назад   5 звезд    
153         Sergey Kojin      2 года назад   5 звезд    
154     Elena Evdokimova      3 года назад   5 звезд    

                                              duration  
0    Главное, ребёнку нравится. Но летом народу оче...  
1                                 Здесь очень весело!)  
2    Очень классное место.Очень весело.Особенно пон...  
3    Замечательный комплекс площадок, много качелей...  
4                             

In [67]:
##остановка драйвера
driver.quit()